# Graph Operations Calculator

In [3]:
import re
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_util
dir(tf.contrib)


/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'batching',
 'bayesflow',
 'checkpoint',
 'cloud',
 'cluster_resolver',
 'coder',
 'compiler',
 'constrained_optimization',
 'copy_graph',
 'crf',
 'cudnn_rnn',
 'data',
 'decision_trees',
 'deprecated',
 'distribute',
 'distributions',
 'eager',
 'estimator',
 'factorization',
 'feature_column',
 'ffmpeg',
 'framework',
 'gan',
 'graph_editor',
 'grid_rnn',
 'image',
 'input_pipeline',
 'integrate',
 'keras',
 'kernel_methods',
 'kfac',
 'labeled_tensor',
 'layers',
 'learn',
 'legacy_seq2seq',
 'linalg',
 'linear_optimizer',
 'lite',
 'lookup',
 'losses',
 'memory_stats',
 'meta_graph_transform',
 'metrics',
 'mixed_precision',
 'model_pruning',
 'nccl',
 'nn',
 'opt',
 'optimizer_v2',
 'periodic_resample',
 'predictor',
 'proto',
 'quantization',
 'quantize',
 'receptive_field',
 'recurrent',
 'reduce_slice_ops',
 'remote_fused_graph',
 'resampler',
 'rnn',

In [29]:
def calculate_metrics(chkpoint):
    """
    Profiles a Tensorflow Graph to obtain the parameters and FLOPS
    params:
    chkpoint: Tensorflow checkpoint. 
    returns: 
        stats: FLOPS proto object 
        params: Parameters proto objects
    
    """
    with tf.Graph().as_default():
        #GPU options. 
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4,
                        visible_device_list='0',
                        allow_growth=True)
        #create session config proto object
        config = tf.ConfigProto(
            gpu_options=gpu_options,
            device_count={
                "GPU": 1,
            },
            allow_soft_placement=True,
            log_device_placement=False
        )
        #Include to initialise variables. 
        run_metadata = tf.RunMetadata()
        # Run session with specified configs.
        with tf.Session(config=config) as sess:
            saver = tf.train.import_meta_graph(chkpoint+'.meta', clear_devices=True)
            saver.restore(sess, chkpoint)
            builder = tf.profiler.ProfileOptionBuilder
            stats = tf.profiler.profile(
                run_meta= run_metadata,
                cmd='op',
                options=tf.profiler.ProfileOptionBuilder.float_operation())
            opts = tf.profiler.ProfileOptionBuilder.trainable_variables_parameter()    
            params = tf.profiler.profile(sess.graph, run_meta=run_metadata, cmd='op', options=opts)
    return stats,params


In [30]:
vox_stats,vox_params = calculate_metrics("/mnt/storage/home/ja17618/scratch/voxelnet/save_model/pre_trained_200/checkpoint-00184673")

INFO:tensorflow:Restoring parameters from /mnt/storage/home/ja17618/scratch/voxelnet/save_model/pre_trained_200/checkpoint-00184673


In [36]:
avod_stats,avod_params= calculate_metrics("/mnt/storage/home/ja17618/scratch/avod/avod/data/outputs/pyramid_cars_example/checkpoints/pyramid_cars_example-00104000")

INFO:tensorflow:Restoring parameters from /mnt/storage/home/ja17618/scratch/avod/avod/data/outputs/pyramid_cars_example/checkpoints/pyramid_cars_example-00104000


In [37]:
vox_params.total_parameters

6661552

In [38]:
print(avod_stats.total_float_ops, avod_params.total_parameters)

696694670729 16682424


In [35]:
print(vox_stats.total_float_ops, vox_params.total_parameters)

1227846793165 6661552


In [24]:
(vox_stats.total_float_ops-avod_stats.total_float_ops)/vox_stats.total_float_ops * 100 

43.258827191856575

In [25]:
(vox_stats.total_float_ops-avod_stats.total_float_ops)

531152122436

In [40]:
(avod_params.total_parameters - vox_params.total_parameters)

10020872

In [39]:
(avod_params.total_parameters - vox_params.total_parameters)/avod_params.total_parameters * 100 

60.06844089324189

In [43]:
vox_stats.total_float_ops

1227846793165

In [45]:
232050483200/vox_stats.total_float_ops*100

18.898977013398177